In [38]:
#@title 🚧 Favor darle "play" a la celda y luego presionar "Ocultar código" 🚧
##@markdown

import ipywidgets as widgets
import os, shutil, zipfile, subprocess, glob, re    # Librerías para gestión de archivos, carpetas y procesos del sistema
from pathlib import Path                            # Herramienta para manipular rutas de archivos de forma intuitiva
import pandas as pd                                 # Librería principal para manipulación y análisis de datos (CSVs)
import numpy as np                                  # Librería para operaciones matemáticas y manejo de arreglos numéricos
import cv2                                          # OpenCV: Librería para procesamiento de imágenes y video
import torch                                        # Framework de Deep Learning para ejecutar el modelo YOLO
from google.colab import files                      # Utilidad específica de Colab para subir y descargar archivos
import random                                       # Permite generar números aleatorios para mezclar datos antes del split
import traceback
from IPython.display import display, HTML, clear_output
from google.colab import output
import io
import contextlib


# Instalaciones de dependencias para el reporte y detección
!pip -q install ultralytics fpdf
from ultralytics import YOLO
from fpdf import FPDF

# --- CONFIGURACIÓN DE RUTAS MAESTRAS ---
BASE = Path("/content")

# Definición de variables globales
DIR_IN = BASE/"in"
DIR_WORK = BASE/"work"
DIR_OUT = BASE/"out"
DIR_DATASET = BASE/"dataset"
DIR_RUNS = BASE/"runs"
DIR_FRAMES = DIR_WORK/"frames"
DIR_RAW = DIR_WORK/"dataset_raw"

# PATHS, para saber que carpetas debe utilizar el entorno
PATHS = {
    "in": DIR_IN,
    "work": DIR_WORK,
    "out": DIR_OUT,
    "frames": DIR_FRAMES,
    "dataset": DIR_DATASET
}

# Creación física de todo el árbol de directorios
for p in [DIR_IN, DIR_WORK, DIR_OUT, DIR_DATASET, DIR_RUNS, DIR_FRAMES, DIR_RAW]:
    p.mkdir(parents=True, exist_ok=True)

# --- CONTENEDOR PRINCIPAL ---
main_container = widgets.VBox()
out = widgets.Output()

STATE = {"paths": PATHS, "out": out}
# ----------------------------------------------------------------------------------------------------------------------------------

def subida_archivos():
    print("📂 Selecciona archivos para añadir (Video, SRT, CSVs o Modelo)")     # Aviso de carga
    subidos = files.upload()                                                   # Selector de archivos

    for nombre_original in subidos.keys():                                     # Itera sobre los archivos subidos
        nombre_min = nombre_original.lower()                                   # Normaliza a minúsculas para comparar

        # --- LÓGICA DE CLASIFICACIÓN (Mantiene archivos existentes) ---
        if "losa" in nombre_min and nombre_original.endswith('.csv'):          # Identifica CSV de fondo
            ruta_destino = PATHS["in"] / "losa.csv"                            # Renombra para el motor de fusión
            print(f"🎯 Identificado como LOSA: {nombre_original}")

        elif "paramento" in nombre_min and nombre_original.endswith('.csv'):   # Identifica CSV de muros
            ruta_destino = PATHS["in"] / "paramento.csv"                       # Renombra para el motor de fusión
            print(f"🎯 Identificado como PARAMENTO: {nombre_original}")

        elif nombre_original.endswith('.pt'):                                 # Identifica modelos YOLO
            ruta_destino = PATHS["in"] / "best.pt"                             # Estandariza nombre del modelo
            print(f"🧠 Modelo cargado/actualizado: {nombre_original}")

        else:                                                                  # Video, SRT y otros
            ruta_destino = PATHS["in"] / nombre_original                       # Conserva nombre original
            print(f"📦 Archivo guardado: {nombre_original}")

        # --- GUARDADO EN DISCO ---
        with open(ruta_destino, "wb") as f:                                    # Abre destino sin borrar el resto de /in
            f.write(subidos[nombre_original])                                  # Escribe el nuevo contenido
    pass

def frames():
    # Busca un video por extensión común (si existe)
    in_files = list(DIR_IN.iterdir())
    video = next((p for p in in_files if p.suffix.lower() in [".mp4", ".mov", ".avi", ".mkv"]), None)  # Encuentra el primer video

    # Extrae frames desde el video detectado (SELECCIONAR MANUALMENTE EN CASO DE VIDEOS MÁS GRANDES)
    fps_extract = 1.0                                              # Define cuántas imágenes por segundo extraer (ej: 1.0 = 1 frame/seg)
    img_ext = "jpg"                                                # Define el formato de imagen de salida (jpg o png)

    # Si no hay video, no hacemos nada y seguimos
    if video is None:                                              # Revisa si se detectó un video en /content/in
        print("⚠️ No hay video detectado en /content/in, así que no se extraen frames todavía.")  # Aviso sin romper el flujo
    else:
        # Limpia frames anteriores para no mezclar corridas
        old_frames = list(DIR_FRAMES.glob(f"*.{img_ext}"))          # Busca frames antiguos en la carpeta de frames
        for f in old_frames:                                        # Recorre frames antiguos
            f.unlink()                                              # Borra cada frame antiguo

        # Define el número inicial desde el cual deseas comenzar la numeración
        start_number = 1262  # Puedes poner el número que desees

        # Define el patrón de nombres con el número de inicio
        out_pattern = str(DIR_FRAMES / f"%06d.{img_ext}")

        # Construye y ejecuta el comando ffmpeg para extraer frames
        cmd = [
        "ffmpeg", "-y",
        "-i", str(video),
        "-vf", f"fps={fps_extract}",
        "-start_number", str(start_number), # <--- ESTO indica dónde empezar
        out_pattern]

        print("✅ Ejecutando FFmpeg para extraer frames:")          # Mensaje de inicio
        print("   ", " ".join(cmd))                                 # Muestra el comando para trazabilidad
        subprocess.run(cmd, check=False)                            # Ejecuta sin romper el notebook si ffmpeg devuelve error

        # Cuenta y muestra resultados
        frames = sorted(DIR_FRAMES.glob(f"*.{img_ext}"))            # Busca los frames generados
        if len(frames) == 0:                                        # Si no se generó ningún frame
            print("⚠️ No se generaron frames (revisa si el video está OK o si fps_extract es muy bajo).")  # Aviso
        else:
            print(f"✅ Frames listos: {len(frames)} en {DIR_FRAMES}")  # Confirma cuántos frames se crearon
            print("   Ejemplo primero/último:", frames[0].name, "|", frames[-1].name)  # Muestra nombres de ejemplo

    # Comprime los frames en un ZIP descargable (si no hay frames, no falla: solo avisa)
    zip_frames_path = DIR_OUT/"frames.zip"                         # Define dónde quedará el zip final

    # Busca frames existentes en la carpeta de frames
    frame_files = sorted(list(DIR_FRAMES.glob("*.jpg")) + list(DIR_FRAMES.glob("*.png")))  # Reúne frames jpg/png

    # Si no hay frames aún, avisa y termina sin error
    if len(frame_files) == 0:                                      # Revisa si hay frames para comprimir
        print("⚠️ No hay frames en /content/work/frames, así que no se puede crear frames.zip todavía.")  # Aviso
    else:
        # Si ya existía un zip antiguo, lo borra para evitar confusiones
        if zip_frames_path.exists():                               # Verifica si el zip ya existe
            zip_frames_path.unlink()                               # Borra el zip anterior

        # Crea el zip con todos los frames
        with zipfile.ZipFile(zip_frames_path, "w", zipfile.ZIP_DEFLATED) as z:  # Abre un zip en modo escritura
            for f in frame_files:                                  # Recorre cada frame
                z.write(f, arcname=f.name)                         # Agrega el archivo al zip con su nombre

        # Confirma que el zip se creó correctamente
        print("✅ ZIP creado para CVAT:", zip_frames_path)          # Muestra la ruta del zip creado
        print(f"✅ Incluye {len(frame_files)} imágenes.")          # Indica cuántas imágenes quedaron dentro

        # Opción de descarga directa (si quieres)
        try:
            from google.colab import files                         # Importa herramienta de descarga de Colab
            files.download(str(zip_frames_path))                   # Descarga el zip a tu PC
            print("✅ Descarga iniciada (si tu navegador lo permite).")  # Confirmación
        except Exception:
            print("⚠️ No pude iniciar descarga automática, pero el zip quedó en /content/out para descargarlo manualmente.")  # Aviso
    pass

def normalizarzip():
    # Descomprime el export de CVAT en /content/work/dataset_raw
    # Nota: Este zip debe llamarse de forma que contenga "cvat" en el nombre

    # Vuelve a buscar un zip que parezca de CVAT
    in_files = list(DIR_IN.iterdir())                                              # Lee archivos en /content/in
    zip_cvat = next((p for p in in_files if p.suffix.lower() == ".zip" and "cvat" in p.name.lower()), None)  # Busca zip con "cvat" en el nombre

    if zip_cvat is None:                                                           # Revisa si existe export de CVAT
        print("⚠️ No detecté ningún .zip de CVAT en /content/in (debe tener 'cvat' en el nombre).")           # Aviso
        print("ℹ️ Cuando lo tengas, súbelo y vuelve a correr este bloque.")         # Guía
        return
    else:
        if DIR_RAW.exists():                                                       # Revisa si ya existe dataset_raw
            shutil.rmtree(DIR_RAW)                                                 # Borra dataset_raw anterior completo
        DIR_RAW.mkdir(parents=True, exist_ok=True)                                 # Crea dataset_raw limpio

        # Descomprime el zip de CVAT dentro de dataset_raw
        with zipfile.ZipFile(zip_cvat, "r") as z:                                  # Abre el zip de CVAT
            z.extractall(DIR_RAW)                                                  # Extrae todo su contenido en DIR_RAW

        # Lista contenido para confirmar que se extrajo algo
        extracted_any = any(DIR_RAW.rglob("*"))                                     # Verifica si hay archivos extraídos
        if not extracted_any:                                                      # Si no se extrajo nada
            print("⚠️ El zip se descomprimió, pero no veo archivos dentro. Revisa si el zip está correcto.")  # Aviso
        else:
            print("✅ Export CVAT descomprimido en:", DIR_RAW)                      # Confirmación
            # Muestra un vistazo rápido de archivos/carpetas extraídas
            top_items = sorted(list(DIR_RAW.iterdir()))                             # Lista el primer nivel de dataset_raw
            print("✅ Primer nivel dentro de dataset_raw:")                         # Título del listado
            for p in top_items[:30]:                                                # Muestra hasta 30 ítems
                tag = "DIR " if p.is_dir() else "FILE"                              # Marca si es carpeta o archivo
                print(" -", tag, p.name)                                            # Imprime nombre
            if len(top_items) > 30:                                                 # Si hay muchos ítems
                print(f" ... y {len(top_items)-30} más")                            # Indica que hay más

    # Detecta imágenes/labels en dataset_raw y construye dataset YOLO final (train/val) creando .txt vacíos para negativos
    import random

    img_exts = {".jpg", ".jpeg", ".png"}                             # Extensiones válidas de imágenes
    train_ratio = 0.8                                                # Proporción train/val
    seed = 42                                                        # Semilla para split repetible

    # Define rutas YOLO estándar de salida
    IMG_TRAIN = DIR_DATASET/"images/train"                           # Imágenes train
    IMG_VAL   = DIR_DATASET/"images/val"                             # Imágenes val
    LBL_TRAIN = DIR_DATASET/"labels/train"                           # Labels train
    LBL_VAL   = DIR_DATASET/"labels/val"                             # Labels val

    # Crea carpetas de salida
    for d in [IMG_TRAIN, IMG_VAL, LBL_TRAIN, LBL_VAL]:               # Recorre carpetas YOLO
        d.mkdir(parents=True, exist_ok=True)                         # Crea si falta

    # Verifica que exista dataset_raw
    if not DIR_RAW.exists():                                         # Si no existe dataset_raw
        print("⚠️ No existe /content/work/dataset_raw todavía. Corre el Bloque 8 (descomprimir CVAT) primero.")  # Aviso
    else:
        # Busca imágenes y txt dentro de dataset_raw (recursivo)
        all_imgs = [p for p in DIR_RAW.rglob("*") if p.suffix.lower() in img_exts]   # Todas las imágenes
        all_txt  = [p for p in DIR_RAW.rglob("*.txt")]                               # Todos los txt

        # Si falta algo, avisa pero no rompe
        if len(all_imgs) == 0:
            print("⚠️ No encontré imágenes dentro de dataset_raw. Revisa estructura del export (o tu zip).")  # Aviso
        if len(all_txt) == 0:
            print("⚠️ No encontré labels (.txt) dentro de dataset_raw. Ojo: CVAT solo exporta txt con anotaciones.")  # Aviso

        # Detecta carpeta con más imágenes y carpeta con más txt (candidatas principales)
        img_dir = None                                               # Carpeta candidata de imágenes
        lbl_dir = None                                               # Carpeta candidata de labels

        if len(all_imgs) > 0:                                        # Si hay imágenes
            counts_img_parent = {}                                   # Conteo por carpeta
            for p in all_imgs:                                       # Recorre imágenes
                counts_img_parent[p.parent] = counts_img_parent.get(p.parent, 0) + 1  # Cuenta
            img_dir = max(counts_img_parent, key=counts_img_parent.get)              # Elige mayor

        if len(all_txt) > 0:                                         # Si hay txt
            counts_lbl_parent = {}                                   # Conteo por carpeta
            for p in all_txt:                                        # Recorre txt
                counts_lbl_parent[p.parent] = counts_lbl_parent.get(p.parent, 0) + 1  # Cuenta
            lbl_dir = max(counts_lbl_parent, key=counts_lbl_parent.get)              # Elige mayor

        print("\n✅ Rutas detectadas (candidatas principales):")      # Imprime rutas
        print(" - img_dir:", str(img_dir) if img_dir else "No detectado")  # Carpeta imágenes
        print(" - lbl_dir:", str(lbl_dir) if lbl_dir else "No detectado")  # Carpeta labels

        # Si no se detectaron rutas, termina sin error
        if (img_dir is None) or (lbl_dir is None):
            print("⚠️ No pude detectar img_dir y/o lbl_dir. Revisa qué hay dentro de dataset_raw (Bloque 4 listar carpetas).")  # Aviso
        else:
            # Crea mapas por stem para hacer match imagen <-> label
            img_map = {}                                             # stem -> ruta imagen
            for p in img_dir.iterdir():                              # Recorre archivos en img_dir
                if p.suffix.lower() in img_exts:                     # Si es imagen
                    img_map[p.stem] = p                              # Guarda

            lbl_map = {}                                             # stem -> ruta label
            for p in lbl_dir.iterdir():                              # Recorre archivos en lbl_dir
                if p.suffix.lower() == ".txt":                       # Si es txt
                    lbl_map[p.stem] = p                              # Guarda

            # Diagnóstico de matching
            all_stems = sorted(img_map.keys())                       # Todas las imágenes (incluye negativos)
            paired = sorted(set(img_map.keys()) & set(lbl_map.keys()))            # Con label
            missing_lbl = sorted(set(img_map.keys()) - set(lbl_map.keys()))      # Sin label (negativos)
            missing_img = sorted(set(lbl_map.keys()) - set(img_map.keys()))      # Txt sin imagen

            print("\n✅ Matching imagen + label (incluyendo negativos):")  # Reporte matching
            print(" - Total imágenes:", len(all_stems))              # Total
            print(" - Con label (.txt):", len(paired))               # Con txt
            print(" - Sin label (se creará .txt vacío):", len(missing_lbl))  # Negativos

            # Si no hay imágenes, no se puede construir dataset
            if len(all_stems) == 0:
                print("⚠️ No hay imágenes válidas para construir dataset. Revisa nombres/extensiones.")  # Aviso
            else:
                # Limpia salidas anteriores
                for d in [IMG_TRAIN, IMG_VAL, LBL_TRAIN, LBL_VAL]:   # Recorre carpetas de salida
                    for f in d.glob("*"):                            # Recorre archivos dentro
                        f.unlink()                                   # Borra

    # ------------------------------------------------------------------------------------------------------------------------------------

    # Split train/val sobre TODAS las imágenes

                random.seed(seed)                                    # Semilla
                random.shuffle(all_stems)                             # Mezcla
                cut = int(len(all_stems) * train_ratio)              # Corte
                train_ids = all_stems[:cut]                          # Train stems
                val_ids = all_stems[cut:]                            # Val stems

                # Copia imagen y label (o crea vacío si falta)
                def copy_img_and_label(stem, img_dst, lbl_dst):      # Función copiar + label
                    img_src = img_map[stem]                          # Imagen fuente
                    shutil.copy2(img_src, img_dst / img_src.name)    # Copia imagen

                    lbl_out = lbl_dst / f"{stem}.txt"                # Label destino
                    if stem in lbl_map:                              # Si existe label real
                        shutil.copy2(lbl_map[stem], lbl_out)         # Copia label
                    else:
                        lbl_out.write_text("", encoding="utf-8")     # Crea label vacío (negativo)

                # Copia a train
                for s in train_ids:                                  # Recorre train
                    copy_img_and_label(s, IMG_TRAIN, LBL_TRAIN)      # Copia

                # Copia a val
                for s in val_ids:                                    # Recorre val
                    copy_img_and_label(s, IMG_VAL, LBL_VAL)          # Copia

                # Resumen final
                n_train_img = len(list(IMG_TRAIN.glob("*")))         # Cuenta imgs train
                n_train_lbl = len(list(LBL_TRAIN.glob("*.txt")))     # Cuenta labels train
                n_val_img = len(list(IMG_VAL.glob("*")))             # Cuenta imgs val
                n_val_lbl = len(list(LBL_VAL.glob("*.txt")))         # Cuenta labels val

                print("\n✅ Dataset YOLO final creado en:", DIR_DATASET)  # Confirma creación
                print(" - Train: imgs =", n_train_img, "| lbls =", n_train_lbl)  # Resumen train
                print(" - Val:   imgs =", n_val_img,   "| lbls =", n_val_lbl)    # Resumen val

                # Chequeo 1 txt por imagen (ideal)
                if n_train_img != n_train_lbl:
                    print("⚠️ Ojo: TRAIN imgs != txt (deberían ser iguales).")  # Aviso
                if n_val_img != n_val_lbl:
                    print("⚠️ Ojo: VAL imgs != txt (deberían ser iguales).")    # Aviso

    # ------------------------------------------------------------------------------------------------------------------------------------

    # Crea el archivo data.yaml para YOLO y DEFINE MANUALMENTE las clases

    yaml_path = BASE/"data.yaml"                                    # Define la ruta donde se guardará el YAML

    # Define aquí tus clases EXACTAS y en el MISMO orden que usaste en CVAT - OJO DEBE HACERSE MANUAL
    classes = [
        "falla junta paramento izquierdo",
        "falla junta paramento derecho",
        "falla junta losa fondo",
        "estría lado izquierdo",
        "estría lado derecho",
        "estría centro",
        "daño paramento",
        "daño losa fondo",
    ]                                                               # Lista de nombres de clases (edítala tú)

    # Verifica que existan carpetas train/val para evitar un YAML apuntando a nada
    train_dir_ok = (DIR_DATASET/"images/train").exists()            # Revisa si existe la carpeta de imágenes train
    val_dir_ok = (DIR_DATASET/"images/val").exists()                # Revisa si existe la carpeta de imágenes val

    # Si no hay estructura dataset, avisa y no rompe el notebook
    if not (train_dir_ok and val_dir_ok):                           # Si faltan carpetas básicas del dataset
        print("⚠️ No detecto la estructura de dataset en /content/dataset/images/train y /val.")  # Aviso
        print("ℹ️ Corre el Bloque 10 (normalización + split) antes de crear el data.yaml.")       # Guía
    else:
        # Si no definiste clases, avisa para que no entrenes con un YAML incompleto
        if len(classes) == 0:                                       # Si la lista de clases está vacía
            print("⚠️ La lista 'classes' está vacía. Agrega tus clases en el orden de CVAT antes de entrenar.")  # Aviso
            print("ℹ️ Igual crearé el data.yaml, pero NO deberías entrenar hasta completar 'classes'.")          # Guía

        # Construye el contenido del YAML que YOLO necesita
        yaml_text = f"""path: {DIR_DATASET}
        train: images/train
        val: images/val
        names:
        """                                                             # Texto base del YAML (path + rutas train/val)

        # Agrega las clases con su índice (0,1,2...) en el orden correcto
        for i, c in enumerate(classes):                              # Recorre clases con índice
            yaml_text += f"  {i}: {c}\n"                              # Agrega cada clase al YAML

        # Guarda el archivo data.yaml
        yaml_path.write_text(yaml_text, encoding="utf-8")            # Escribe el YAML en disco

        # Prints de confirmación + vista rápida del contenido
        print("✅ data.yaml creado en:", yaml_path)                   # Confirma ruta del archivo creado
        print("✅ Contenido de data.yaml:")                           # Título del contenido
        print(yaml_text)                                              # Muestra el texto completo
    pass

def entrenar():
    from google.colab import files  # Necesario para la descarga automática

    # 1. Configuración de parámetros
    data_yaml = "/content/data.yaml"
    batch_size = 16
    epochs = 215
    img_size = 640
    output_dir = "/content/runs"
    best_pt_path = Path("/content/in/best.pt") # Ruta de tu "Partida Guardada"

    # Asegurar que la carpeta /content/in existe
    best_pt_path.parent.mkdir(parents=True, exist_ok=True)

    # 2. Lógica de carga de modelo (Continuar Partida o Empezar de Cero)
    if best_pt_path.exists():
        print(f"🎮 'Save Game' detectado. Cargando progreso desde: {best_pt_path}")
        model = YOLO(str(best_pt_path))
        lr_inicial = 0.001 # Tasa más baja para no arruinar lo ya aprendido
    else:
        print("🆕 No hay partida guardada. Iniciando entrenamiento desde cero (YOLOv8n).")
        model = YOLO("yolov8n.pt")
        lr_inicial = 0.01

    # 3. Entrenamiento
    print(f"🚀 Iniciando entrenamiento (Máximo {epochs} epochs)...")
    results = model.train(
        data=data_yaml,
        epochs=epochs,
        imgsz=img_size,
        batch=batch_size,
        project=output_dir,
        name="train_experiment",
        exist_ok=True,
        lr0=lr_inicial,
        patience=30,      # Si deja de mejorar por 30 epochs, se detiene
        save=True,
        pretrained=True
    )

    # 4. Gestión de archivos y Auto-Descarga ("Guardado Final")
    print("✅ Entrenamiento finalizado.")

    # Ruta donde YOLO acaba de guardar el mejor modelo de esta sesión
    new_best_path = Path(output_dir) / "train_experiment" / "weights" / "best.pt"

    if new_best_path.exists():
        # 4a. Sobreescribir el Save Game en /content/in (Para la próxima vez que corras el bloque)
        shutil.copy2(new_best_path, best_pt_path)
        print(f"⭐ 'Save Game' actualizado localmente en: {best_pt_path}")

        # 4b. Descargar el archivo al PC automáticamente
        try:
            print("📥 Iniciando descarga del modelo 'best.pt' a tu ordenador...")
            files.download(str(best_pt_path))
            print("✅ Descarga iniciada. ¡Guarda bien este archivo!")
        except Exception as e:
            print(f"⚠️ No se pudo iniciar la descarga automática: {e}")
            print("ℹ️ Puedes descargarlo manualmente desde la carpeta /content/in en el panel izquierdo.")
    else:
        print("⚠️ Error: No se encontró el archivo generado en esta sesión.")
    pass

def validar():
    # --- 1. CONFIGURACIÓN DE RUTAS ---
    base_path = Path("/content")
    dir_dataset = base_path / "dataset"
    dir_runs = base_path / "runs"
    dir_out = base_path / "out"
    dir_in = base_path / "in"

    # Ruta del modelo y de las imágenes
    best_model_path = dir_in / "best.pt"
    val_images_path = dir_dataset / "images/val"

    # Carpeta de salida
    pred_name = "predict_val"
    final_pred_dir = dir_runs / pred_name

    # --- 2. VALIDACIÓN ---
    if not best_model_path.exists():
        print(f"⚠️ No se encuentra el modelo en {best_model_path}. Revisa el Bloque 12.")
    elif not val_images_path.exists():
        print(f"⚠️ No existe la carpeta de validación en {val_images_path}")
    else:
        try:
            print(f"🚀 Iniciando predicción visual con líneas delgadas...")
            model = YOLO(str(best_model_path))

            # Ejecutamos la predicción con ajustes visuales
            model.predict(
                source=str(val_images_path),
                save=True,
                project=str(dir_runs),
                name=pred_name,
                exist_ok=True,
                conf=0.25,         # Solo muestra lo que tenga > 25% certeza
                iou=0.3,          # Umbral de solapamiento (ajustar si hay cajas duplicadas)
                # --- AJUSTES PARA QUE NO SE TAPEN LAS FALLAS ---
                line_width=3,
                show_labels=True,  # Muestra el nombre de la falla
                show_conf=False,   # Quitamos el % de confianza para limpiar la imagen
                save_txt=False     # No necesitamos los .txt aquí, solo las fotos
            )

            print(f"✅ Fotos guardadas en: {final_pred_dir}")

            # --- 3. CREACIÓN DE ZIP Y DESCARGA ---
            zip_file_path = dir_out / "val_predictions_clean.zip"
            dir_out.mkdir(parents=True, exist_ok=True)

            # Empaquetamos solo las imágenes resultantes
            with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
                # Buscamos extensiones comunes de imagen
                for ext in ['*.jpg', '*.jpeg', '*.png']:
                    for img_file in final_pred_dir.rglob(ext):
                        zipf.write(img_file, arcname=img_file.name)

            if zip_file_path.stat().st_size > 0:
                print(f"📦 ZIP creado: {zip_file_path}")
                files.download(str(zip_file_path))
                print("📥 Descarga iniciada.")
            else:
                print("⚠️ El ZIP está vacío. ¿Seguro que hubo detecciones?")

        except Exception as e:
            print(f"❌ Error: {e}")
    pass

def detectar():
    # --- VALIDACIÓN PREVIA DE ARCHIVOS CRÍTICOS ---
    archivos_faltantes = []

    # 1. Verificar Video (.mp4)
    video_check = list(PATHS["in"].glob("*.mp4"))
    if not video_check: archivos_faltantes.append("Video (.mp4)")
    else: print(f"✅ Video detectado: {video_check[0].name}")

    # 2. Verificar Modelo (best.pt)
    modelo_path = PATHS["in"] / "best.pt"
    if not modelo_path.exists(): archivos_faltantes.append("Modelo (best.pt)")
    else: print(f"✅ Modelo detectado: best.pt")

    # 3. Verificar Telemetría (.srt)
    srt_check = list(PATHS["in"].glob("*.srt"))
    if not srt_check: archivos_faltantes.append("Telemetría (.srt)")
    else: print(f"✅ SRT detectado: {srt_check[0].name}")

    # 4. Verificar CSV Losa
    losa_path = PATHS["in"] / "losa.csv"
    if not losa_path.exists(): archivos_faltantes.append("CSV de Losa (losa.csv)")
    else: print(f"✅ Datos de Losa detectados")

    # 5. Verificar CSV Paramento
    paramento_path = PATHS["in"] / "paramento.csv"
    if not paramento_path.exists(): archivos_faltantes.append("CSV de Paramento (paramento.csv)")
    else: print(f"✅ Datos de Paramento detectados")

    # --- CONTROL DE DETENCIÓN
    if archivos_faltantes:
        print("\n❌ ERROR: Faltan los siguientes archivos en la carpeta de entrada:")
        for arch in archivos_faltantes:
            print(f"   - {arch}")
        raise SystemExit("Deteniendo ejecución: Faltan requisitos para procesar el reporte.")
    else:
        print("\n🚀 Todos los componentes listos. Iniciando procesamiento...\n" + "-"*50)

    # ----------------------------------------------------------------------------------------------------------------------------------

    # Analiza los archivos SRT para datos georeferenciales y CSVs para datos técnicos

    def procesar_telemetria_srt(ruta_srt):
        print(f"📖 Leyendo archivo: {ruta_srt.name}")
        with open(ruta_srt, 'r', encoding='utf-8-sig', errors='ignore') as f:
            contenido = f.read()

        # Captura tiempos como "0:00:01" o "00:00:01,000"
        patron_tiempo = r'(\d{1,2}:\d{1,2}:\d{1,2}(?:[.,]\d+)?) --> (\d{1,2}:\d{1,2}:\d{1,2}(?:[.,]\d+)?)'
        tiempos = re.findall(patron_tiempo, contenido)

        # 2. Dividimos el contenido por las marcas de tiempo para obtener los textos
        partes = re.split(patron_tiempo, contenido)
        # Tras el split, los textos quedan en las posiciones 3, 6, 9... (saltando los grupos de captura)
        textos = partes[3::3]

        mapeo = []

        for i, (inicio, fin) in enumerate(tiempos):
            try:
                # Procesar el tiempo de inicio a segundos totales
                partes_hms = inicio.replace(',', '.').split(':')
                h = int(partes_hms[0])
                m = int(partes_hms[1])
                s = float(partes_hms[2])
                seg_total = h * 3600 + m * 60 + s

                # Obtener el texto del bloque actual
                texto_bloque = textos[i] if i < len(textos) else ""

                # Limpiar etiquetas <i> y sacar el último número (el metro)
                texto_limpio = re.sub(r'<.*?>', '', texto_bloque).strip()
                numeros = re.findall(r"[-+]?\d*\.\d+|\d+", texto_limpio)

                if numeros:
                    # Tomamos el último número porque tu SRT dice "TRAMO 2 - Parte 2 - 1343 m"
                    # El 1343 es el que nos interesa.
                    m_valor = float(numeros[-1])
                    mapeo.append({'s': seg_total, 'm': m_valor})
            except Exception as e:
                continue

        df_map = pd.DataFrame(mapeo)
        return df_map

    print(f"✅ SRT leído correctamente") # Confirmación de lectura

    # --- CONFIGURACIÓN DE UMBRAL DE DATOS TÉCNICOS (CSV) ---
    UMBRAL_DISTANCIA = 2.0                                              # Límite de distancia para validar datos (+-2m)

    def cargar_datos_robot(ruta):                                       # Función para leer CSVs del robot
        try:
            df = pd.read_csv(ruta, sep=';', decimal=',', encoding='latin-1') # Intenta lectura con codificación Latin-1
        except:
            df = pd.read_csv(ruta, sep=';', decimal=',', encoding='cp1252') # Si falla, intenta con CP1252
        return df.loc[:, ~df.columns.str.contains('^Unnamed')]          # Elimina columnas vacías residuales

    print("🧠 Fusionando datos con regla de proximidad estricta (+-2m)...")
    df_l = cargar_datos_robot(PATHS["in"] / "losa.csv")                 # Carga base de datos de losa
    df_p = cargar_datos_robot(PATHS["in"] / "paramento.csv")            # Carga base de datos de paramento
    print(f"✅ CSVs leídos correctamente: losa ({len(df_l)} filas), paramento ({len(df_p)} filas)") # Confirmación de lectura

    datos_finales = []                                                  # Lista para almacenar resultados procesados

    # ----------------------------------------------------------------------------------------------------------------------------------

    # Analiza el video usando el Modelo DAICH y se ubica espacialmente usando el SRT

    # Modelo YOLO analiza video y detecta clases
    model = YOLO(PATHS["in"] / "best.pt")
    video_path = next(PATHS["in"].glob("*.mp4"))
    df_telemetria = procesar_telemetria_srt(next(PATHS["in"].glob("*.srt")))

    df_vis = pd.DataFrame()
    if df_telemetria.empty:
        print("❌ ABORTANDO: El archivo SRT no tiene el formato correcto.")
    else:
        cap = cv2.VideoCapture(str(video_path))
        fps = cap.get(cv2.CAP_PROP_FPS)
        hallazgos_crudos = []
        f_count = 0

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret: break
            if f_count % int(fps) == 0:
                seg_act = f_count / fps
                m_act = np.interp(seg_act, df_telemetria['s'], df_telemetria['m'])
                preds = model.predict(frame, conf=0.25, verbose=False)
                for r in preds:
                    if len(r.boxes) > 0:
                        img_f = f"evidencia_m_{m_act:.2f}.jpg"
                        cv2.imwrite(str(PATHS["frames"] / img_f), r.plot())
                        for b in r.boxes:
                            hallazgos_crudos.append({
                                'metro': m_act,
                                'seg_video': seg_act, # <--- GUARDAMOS EL SEGUNDO
                                'cls': model.names[int(b.cls)],
                                'img': img_f
                            })
                print(f"🔍 Escaneando Metro: {m_act:.2f} | Detecciones: {len(hallazgos_crudos)}", end="\r")
            f_count += 1
        cap.release()
        df_vis = pd.DataFrame(hallazgos_crudos)
        print(f"\n✅ Análisis visual terminado. Se encontraron {len(df_vis)} registros.")

    # ----------------------------------------------------------------------------------------------------------------------------------

    # Relaciona las detecciones del video con los datos técnicos disponibles (CSVs)

    for _, det in df_vis.iterrows():                                    # Itera por cada detección de la IA
        clase_ia = det['cls'].lower()                                   # Convierte clase a minúsculas para comparar

        # Selección de DB según palabras clave en la clase detectada
        db, origen = (df_l, "losa.csv") if any(k in clase_ia for k in ["estría", "losa", "fondo"]) else (df_p, "paramento.csv")

        db['diff'] = (db['Metros'] - det['metro']).abs()                # Calcula distancia absoluta entre IA y Robot
        cercano = db.nsmallest(1, 'diff').iloc[0]                       # Encuentra el registro más cercano (sin interpolar)
        distancia_minima = cercano['diff']                              # Guarda la distancia del punto más próximo

        res = {**det, 'csv_usado': origen}                              # Crea diccionario con datos base y origen CSV
        cols_tecnicas = ['Extensión (cm)', 'Alto (cm)', 'Área (m2)', 'Volumen (m3)', 'Profundidad (cm)']

        if distancia_minima <= UMBRAL_DISTANCIA:                        # VALIDACIÓN: Si está dentro del rango de 2 metros
            res['tiene_datos'] = True                                   # Marca como hallazgo validado técnicamente
            for c in cols_tecnicas:
                res[c] = cercano.get(c, 0)                              # Asigna valores técnicos del robot
            res['Magnitud'] = cercano.get('Magnitud', 'N/A')            # Captura magnitud de daño
        else:                                                           # Si está fuera de rango (ej. 1343 vs 1336)
            res['tiene_datos'] = False                                  # Marca para omitir datos en el PDF
            for c in cols_tecnicas: res[c] = 0                          # Setea valores técnicos en cero
            res['Magnitud'] = "N/A"                                     # Magnitud no disponible

        datos_finales.append(res)                                       # Agrega el resultado a la lista final

    df_temp = pd.DataFrame(datos_finales)                               # Convierte resultados a DataFrame temporal
    df_temp['metro_redondo'] = df_temp['metro'].round(1)                # Redondea metro a 10cm para agrupar cámaras
    # Elimina duplicados si coinciden metro y clase (limpieza de doble cámara)
    df_reporte = df_temp.drop_duplicates(subset=['metro_redondo', 'cls'], keep='first').copy()

    print(f"📊 Fusión lista. Detecciones totales a reportar: {len(df_reporte)}") # Resumen final del proceso

    # ----------------------------------------------------------------------------------------------------------------------------------

    # Construye el documento PDF con las evidencias capturadas y los datos técnicos

    print("📄 Generando reporte final...")
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)

    def clean(val):
        try:
            f = float(val)
            return ('%.4f' % f).rstrip('0').rstrip('.')
        except: return str(val)

    for idx, f in df_reporte.iterrows():
        pdf.add_page()

        # --- TÍTULO: CLASE
        pdf.set_font("Arial", 'B', 14)

        clase_nombre = f['cls'].upper()
        # Lógica de género para el sufijo
        sufijo = "DETECTADO/A"

        # Redondeo de la cifra original a 2 decimales
        metro_tit = f"{round(f['metro'], 2):.2f}"

        # Construcción del título en mayúsculas
        titulo_final = f"{clase_nombre} {sufijo} EN KM {metro_tit}"
        pdf.cell(200, 10, titulo_final, ln=True, align='C')

        # Imagen de la IA
        pdf.image(str(PATHS["frames"]/f['img']), x=10, y=30, w=180)

        # --- SUBTÍTULO: Solo Tiempo y Origen ---
        pdf.set_y(155)
        pdf.set_font("Arial", 'B', 10)
        pdf.set_fill_color(230, 230, 230)

        m, s = divmod(int(f['seg_video']), 60)
        info_sub = f" TIEMPO VIDEO: {m}m {s}s | ORIGEN DATOS: {f['csv_usado']}"
        pdf.cell(190, 10, info_sub, ln=True, fill=True)

        pdf.set_font("Arial", '', 10)

        if f['tiene_datos']:
            # Fila 1: Extensión y Ancho
            pdf.cell(95, 8, f" Extensión: {clean(f.get('Extensión (cm)', 0))} cm", border=1)
            pdf.cell(95, 8, f" Ancho: {clean(f.get('Alto (cm)', 0))} cm", border=1, ln=True)

            # Fila 2: Área m² y Volumen m³
            area = f.get('Área (m2)', f.get('Area (m2)', 0))
            pdf.cell(95, 8, f" Área: {clean(area)} m²", border=1)
            pdf.cell(95, 8, f" Volumen: {clean(f.get('Volumen (m3)', 0))} m³", border=1, ln=True)

            # Fila 3: Profundidad y Magnitud
            pdf.cell(95, 8, f" Profundidad: {clean(f.get('Profundidad (cm)', 0))} cm", border=1)
            pdf.cell(95, 8, f" Magnitud de daño: {f.get('Magnitud', 'N/A')}", border=1, ln=True)
        else:
            # Nota roja
            pdf.set_text_color(200, 0, 0)
            pdf.set_font("Arial", 'B', 10)
            msj = "\nAVISO: ESTE HALLAZGO DETECTADO POR EL MODELO NO TIENE COINCIDENCIA TÉCNICA EN LA NUBE DE PUNTOS DEL ROBOT (+-2M)."
            pdf.multi_cell(190, 8, msj, border=1, align='C')
            pdf.set_text_color(0, 0, 0)

    report_name = "Reporte_Inspeccion_Final.pdf"
    pdf.output(report_name)
    files.download(report_name)
    pass

In [45]:
# =========================================================
# UI PRO (sin ipywidgets) — HTML + callbacks (Colab compatible)
# PEGAR COMPLETO DESPUÉS DEL BLOQUE MOTOR:
# subida_archivos, frames, normalizarzip, entrenar, validar, detectar
# =========================================================

from IPython.display import display, HTML, clear_output
from google.colab import output
import io
import contextlib
import traceback
import time
import logging
from pathlib import Path

# -----------------------------
# Helpers: presencia de archivos
# -----------------------------
VIDEO_EXTS = [".mp4", ".mov", ".avi", ".mkv"]

def _find_first_video():
    p_in = PATHS["in"]
    for ext in VIDEO_EXTS:
        vids = list(p_in.glob(f"*{ext}"))
        if vids:
            return vids[0]
    return None

def _presence():
    p_in = PATHS["in"]
    return {
        "best": (p_in/"best.pt") if (p_in/"best.pt").exists() else None,
        "video": _find_first_video(),
        "srt": (list(p_in.glob("*.srt"))[0] if list(p_in.glob("*.srt")) else None),
        "losa": (p_in/"losa.csv") if (p_in/"losa.csv").exists() else None,
        "paramento": (p_in/"paramento.csv") if (p_in/"paramento.csv").exists() else None,
        "cvat": (next((z for z in p_in.glob("*.zip") if "cvat" in z.name.lower()), None)),
    }

def _missing(required_keys):
    pres = _presence()
    return [k for k in required_keys if pres.get(k) is None]

def _dataset_ready():
    train_dir = PATHS["dataset"]/ "images/train"
    val_dir   = PATHS["dataset"]/ "images/val"
    return (Path("/content/data.yaml").exists()
            and train_dir.exists() and val_dir.exists()
            and len(list(train_dir.glob("*"))) > 0
            and len(list(val_dir.glob("*"))) > 0)

def _status_dict():
    pres = _presence()
    frames_count = len(list(PATHS["frames"].glob("*.jpg"))) + len(list(PATHS["frames"].glob("*.png")))

    train_dir = PATHS["dataset"]/ "images/train"
    val_dir   = PATHS["dataset"]/ "images/val"
    train_imgs = len(list(train_dir.glob("*"))) if train_dir.exists() else 0
    val_imgs   = len(list(val_dir.glob("*"))) if val_dir.exists() else 0
    dataset_ok = (train_imgs > 0 and val_imgs > 0)
    yaml_ok = Path("/content/data.yaml").exists()

    return {
        "best": pres["best"].name if pres["best"] else None,
        "video": pres["video"].name if pres["video"] else None,
        "srt": pres["srt"].name if pres["srt"] else None,
        "losa": pres["losa"].name if pres["losa"] else None,
        "paramento": pres["paramento"].name if pres["paramento"] else None,
        "cvat": pres["cvat"].name if pres["cvat"] else None,
        "frames_count": frames_count,
        "train_imgs": train_imgs,
        "val_imgs": val_imgs,
        "dataset_ok": dataset_ok,
        "yaml_ok": yaml_ok
    }

LABELS = {
    "best": "Modelo (best.pt)",
    "video": "Video",
    "srt": "Telemetría (SRT)",
    "losa": "CSV de losa (losa.csv)",
    "paramento": "CSV de paramento (paramento.csv)",
    "cvat": "Export de CVAT (ZIP que contenga 'cvat' en el nombre)"
}

def _missing_text(required_keys):
    miss = _missing(required_keys)
    if not miss:
        return "Ninguno (todo OK)."
    return ", ".join([LABELS[k] for k in miss])

# -----------------------------
# UI state
# -----------------------------
_UI = {
    "screen": "welcome",
    "nav": ["welcome"],
    "log": "",
    "training_ready": False,     # True cuando ya corrió normalizarzip
    "last_download": None,       # path str del último archivo descargable
    "last_download_label": None  # nombre amigable
}

# -----------------------------
# Log resumido
# -----------------------------
def _pretty_log(text, max_lines=70):
    lines = (text or "").splitlines()
    if len(lines) <= max_lines:
        return text or ""
    head = lines[:10]
    tail = lines[-(max_lines-12):]
    return "\n".join(head + ["", "… (log resumido) …", ""] + tail)

# -----------------------------
# Silenciar logs feos de Ultralytics (los "image 1/253 ...")
# -----------------------------
class _SilenceUltralytics:
    def __enter__(self):
        self.saved = []
        for name in ["ultralytics", "ultralytics.yolo", "yolo", ""]:
            lg = logging.getLogger(name)
            self.saved.append((lg, lg.level, lg.disabled))
            lg.setLevel(logging.ERROR)
            lg.disabled = False
        root = logging.getLogger()
        self.root_saved = (root, root.level)
        root.setLevel(logging.ERROR)
        return self

    def __exit__(self, exc_type, exc, tb):
        for lg, level, disabled in self.saved:
            try:
                lg.setLevel(level)
                lg.disabled = disabled
            except:
                pass
        try:
            root, lvl = self.root_saved
            root.setLevel(lvl)
        except:
            pass

# -----------------------------
# Render HTML
# -----------------------------
def _requirements_html(required_keys, title):
    pres = _presence()
    rows = []
    for k in required_keys:
        ok = pres.get(k) is not None
        icon = "✅" if ok else "❌"
        color = "#16a34a" if ok else "#dc2626"
        fname = pres[k].name if ok else "FALTANTE"
        rows.append(f"""
          <div class="reqrow">
            <span style="color:{color}; font-weight:900;">{icon}</span>
            <span class="reqlabel">{LABELS[k]}</span>
            <span class="reqfile"><b>{fname}</b></span>
          </div>
        """)

    missing = _missing(required_keys)
    head = "✅ Requisitos completos" if not missing else "❌ Faltan requisitos"
    head_color = "#86efac" if not missing else "#fecaca"

    return f"""
      <div class="card">
        <div class="cardtitle">{title}</div>
        <div class="subbox">
          <div style="font-weight:900; color:{head_color}; margin-bottom:8px;">{head}</div>
          {''.join(rows)}
        </div>
      </div>
    """

def _chip(ok, label, detail):
    icon = "✅" if ok else "❌"
    color = "#16a34a" if ok else "#dc2626"
    bg = "#062a12" if ok else "#2a0a0a"
    d = detail if detail else "no disponible"
    return f"""
      <div class="chip" style="border-color:{color}; background:{bg};">
        <div class="chipttl">{icon} {label}</div>
        <div class="chipdet">{d}</div>
      </div>
    """

def _status_html():
    s = _status_dict()
    return f"""
      <div class="card">
        <div class="cardtitle">Estado del sistema</div>
        <div class="grid4">
          {_chip(bool(s["best"]), "Modelo best.pt", s["best"])}
          {_chip(bool(s["video"]), "Video", s["video"])}
          {_chip(bool(s["srt"]), "SRT", s["srt"])}
          {_chip(bool(s["cvat"]), "CVAT ZIP", s["cvat"])}
        </div>
        <div style="height:10px"></div>
        <div class="grid4">
          {_chip(bool(s["losa"]), "CSV losa", s["losa"] or "losa.csv")}
          {_chip(bool(s["paramento"]), "CSV paramento", s["paramento"] or "paramento.csv")}
          {_chip((s["frames_count"]>0), "Frames", f'{s["frames_count"]} imágenes')}
          {_chip(bool(s["dataset_ok"]), "Dataset YOLO", f'train {s["train_imgs"]} | val {s["val_imgs"]}')}
        </div>
        <div style="height:10px"></div>
        <div class="grid4">
          {_chip(bool(s["yaml_ok"]), "data.yaml", "disponible" if s["yaml_ok"] else "no disponible")}
        </div>
      </div>
    """

def _btn(label, action, style="primary", disabled=False):
    dis = "disabled" if disabled else ""
    return f"""<button class="btn {style}" {dis} onclick="daichAction('{action}')">{label}</button>"""

def _topbar(title):
    can_back = len(_UI["nav"]) > 1
    back_btn = _btn("⬅️ Atrás", "back", "ghost", disabled=(not can_back))
    return f"""
      <div class="topbar">
        <div style="width:160px">{back_btn}</div>
        <div class="toptitle">{title}</div>
      </div>
    """

def _wrap(inner):
    return f"""
    <style>
      .wrap {{
        font-family: ui-sans-serif, system-ui, -apple-system, Segoe UI, Roboto, Arial;
        color: #e5e7eb;
      }}
      .card {{
        background:#0b0f19; border:1px solid #1f2a44; border-radius:18px;
        padding:16px 18px; margin-bottom:12px;
      }}
      .bigtitle {{ font-size:20px; font-weight:1000; }}
      .subtitle {{ color:#94a3b8; margin-top:6px; font-size:13px; line-height:1.35; }}
      .cardtitle {{ font-size:14px; font-weight:1000; margin-bottom:10px; color:#e5e7eb; }}
      .p {{ color:#cbd5e1; font-size:13px; line-height:1.55; }}
      .center {{ display:flex; justify-content:center; }}
      .grid1 {{ display:grid; grid-template-columns: 1fr; gap:10px; }}
      .grid2 {{ display:grid; grid-template-columns: 1fr 1fr; gap:10px; }}
      .grid4 {{ display:grid; grid-template-columns: repeat(4, 1fr); gap:10px; }}
      .row {{ display:flex; gap:10px; flex-wrap:wrap; }}

      .btn {{
        width:100%; padding:14px 14px; border-radius:14px; border:1px solid #2b3a5a;
        background:#111827; color:#e5e7eb; cursor:pointer; font-weight:1000;
        font-size:14px;
      }}
      .btn:hover {{ filter: brightness(1.08); }}
      .btn:disabled {{ opacity:.45; cursor:not-allowed; }}
      .btn.success {{ background:#064e3b; border-color:#10b981; }}
      .btn.danger {{ background:#3f0a0a; border-color:#ef4444; }}
      .btn.warning {{ background:#3b2a06; border-color:#f59e0b; }}
      .btn.info {{ background:#072a3f; border-color:#38bdf8; }}
      .btn.primary {{ background:#0b1a3a; border-color:#60a5fa; }}
      .btn.ghost {{ background:transparent; border-color:#2b3a5a; }}

      .topbar {{
        display:flex; align-items:center; gap:12px; margin-bottom:12px;
      }}
      .toptitle {{ font-weight:1000; font-size:16px; }}

      .subbox {{
        background:#111827; border:1px solid #26324d; border-radius:14px; padding:12px;
      }}
      .reqrow {{
        display:flex; align-items:center; gap:8px; padding:6px 0;
        border-bottom: 1px solid rgba(38,50,77,.35);
      }}
      .reqrow:last-child {{ border-bottom:none; }}
      .reqlabel {{ color:#cbd5e1; flex: 1; }}
      .reqfile {{ color:#e5e7eb; }}

      .chip {{
        border:1px solid #2b3a5a; border-radius:14px; padding:10px 12px;
      }}
      .chipttl {{ font-weight:1000; font-size:13px; }}
      .chipdet {{ opacity:.85; font-size:12px; margin-top:4px; }}

      .logbox {{
        height: 130px; overflow:auto; background:#0a1020;
        border:1px solid #26324d; border-radius:14px; padding:10px;
      }}
      pre {{ margin:0; white-space: pre-wrap; word-break: break-word; color:#cbd5e7; }}
    </style>

    <div class="wrap">
      <script>
        function daichAction(action) {{
          google.colab.kernel.invokeFunction('daich_ui_action', [action], {{}});
        }}
      </script>

      {inner}
    </div>
    """

# -----------------------------
# AYUDA DE USO (REEMPLAZADA por tu versión extensa)
# -----------------------------
def _help_html():
    return f"""
      {_topbar("Ayuda de uso")}

      <div class="card">
        <div class="bigtitle">📘 DAICH — Guía de uso</div>
        <div class="subtitle">
          Una explicación simple, completa y sin tecnicismos para entender qué hace cada cosa.
        </div>
      </div>

      <div class="card">
        <div class="cardtitle">¿Qué es DAICH y por qué existe?</div>
        <div class="p">
          DAICH es una interfaz pensada para inspeccionar recorridos (video + telemetría) y generar un reporte con hallazgos,
          y también para mejorar el modelo con nuevos ejemplos (entrenamiento).<br><br>
          La idea es simple: en terreno se producen videos y datos, pero revisarlos manualmente toma mucho tiempo y es fácil pasar cosas por alto.
          DAICH busca que el proceso sea más rápido, consistente y que el sistema pueda aprender con el tiempo cuando se le agregan nuevas anotaciones.
        </div>
      </div>

      <div class="card">
        <div class="cardtitle">Modo detección (para generar un reporte)</div>
        <div class="p">
          Este modo sirve para analizar un video y producir un PDF con evidencias y datos asociados.<br><br>

          <b>¿Qué se necesita?</b><br>
          • <b>best.pt</b>: el modelo ya entrenado.<br>
          • <b>Video</b>: el recorrido a inspeccionar.<br>
          • <b>SRT</b>: telemetría del video (para ubicar detecciones en el recorrido).<br>
          • <b>losa.csv</b> y <b>paramento.csv</b>: datos técnicos para cruzar con la detección.<br><br>

          <b>¿Cómo se usa?</b><br>
          1) Entrar a <b>Modo detección</b>.<br>
          2) El sistema pedirá lo que falte.<br>
          3) Cuando esté todo, se ejecuta y descarga el reporte PDF.
        </div>
      </div>

      <div class="card">
        <div class="cardtitle">Modo entrenamiento (para mejorar el modelo)</div>
        <div class="p">
          Este modo es para cuando se quiere que el sistema aprenda con nuevos datos.<br><br>

          <b>Flujo recomendado</b><br>
          1) Cargar <b>best.pt</b> (modelo base).<br>
          2) <b>Video → Frames</b>: se extraen imágenes del video para anotarlas.<br>
          3) <b>Anotar en CVAT</b>: se sube el ZIP de exportación de CVAT.<br>
          4) <b>Preparar dataset</b>: DAICH arma el dataset YOLO y el data.yaml.<br>
          5) <b>Entrenar</b>: el modelo se actualiza con lo nuevo.<br>
          6) <b>Validar</b>: se generan imágenes de validación y se descargan automáticamente en ZIP.
        </div>
      </div>

      <div class="card">
        <div class="cardtitle">¿Qué ganas con esto?</div>
        <div class="p">
          • El modelo se vuelve más robusto para casos reales.<br>
          • Se corrigen errores típicos agregando ejemplos nuevos.<br>
          • Se reduce trabajo repetitivo en futuras inspecciones.
        </div>
      </div>
    """

def _screen_html():
    scr = _UI["screen"]

    log_html = f"""
      <div class="card">
        <div class="cardtitle">Panel de acciones</div>
        <div class="logbox"><pre>{_UI["log"] or "Sin acciones recientes."}</pre></div>
      </div>
    """

    if scr == "welcome":
        body = f"""
          <div class="card">
            <div class="bigtitle">🤖 DAICH — Panel de Control</div>
            <div class="subtitle">Sistema de inspección y entrenamiento con dos modos de operación.</div>
          </div>

          <div class="card">
            <div class="cardtitle">Inicio rápido</div>
            <div class="p">
              Presiona <b>¡Comencemos!</b> y elige el modo. La interfaz te dirá qué falta y qué se puede ejecutar.
            </div>
          </div>

          <div class="center">
            {_btn("🚀 ¡Comencemos!", "go_home", "success")}
          </div>
        """
        return _wrap(body)

    if scr == "home":
        dl_ok = bool(_UI["last_download"] and Path(_UI["last_download"]).exists())
        body = f"""
          {_topbar("Menú principal")}

          <div class="card">
            <div class="bigtitle">🧭 Menú principal</div>
            <div class="subtitle">Selecciona el modo. El estado del sistema se ve solo si lo pides.</div>
          </div>

          <div class="grid1">
            {_btn("1️⃣ Modo detección", "go_detection", "danger")}
            {_btn("2️⃣ Modo entrenamiento", "go_training", "warning")}
            {_btn("3️⃣ Ver estado del sistema", "go_status", "info")}
            {_btn("4️⃣ Ayuda de uso", "go_help", "primary")}
          </div>

          <div class="grid1" style="margin-top:10px;">
            {_btn(f"⬇️ Descargar último resultado" + (f" ({_UI['last_download_label']})" if _UI['last_download_label'] else ""), "dl_last", "success", disabled=(not dl_ok))}
          </div>

          {log_html}
        """
        return _wrap(body)

    if scr == "help":
        body = _help_html() + log_html
        return _wrap(body)

    if scr == "status":
        body = f"""
          {_topbar("Estado del sistema")}
          {_status_html()}
          {log_html}
        """
        return _wrap(body)

    if scr == "detection":
        required = ["best", "video", "srt", "losa", "paramento"]
        can_run = (len(_missing(required)) == 0)

        body = f"""
          {_topbar("Modo detección")}

          <div class="card">
            <div class="cardtitle">📌 Faltantes</div>
            <div class="p"><b>{_missing_text(required)}</b></div>
          </div>

          {_requirements_html(required, "📥 Checklist detección")}

          <div class="grid1">
            {_btn("📤 Subir archivos", "det_upload", "primary")}
          </div>

          <div class="grid1" style="margin-top:10px;">
            {_btn("▶️ Ejecutar detección", "det_run", "success", disabled=(not can_run))}
          </div>

          {log_html}
        """
        return _wrap(body)

    if scr == "training_model":
        req_model = ["best"]
        can_continue = (len(_missing(req_model)) == 0)

        body = f"""
          {_topbar("Modo entrenamiento")}

          <div class="card">
            <div class="cardtitle">📌 Faltantes</div>
            <div class="p"><b>{_missing_text(req_model)}</b></div>
          </div>

          {_requirements_html(req_model, "📥 Checklist modelo")}

          <div class="grid1">
            {_btn("📤 Subir best.pt", "tr_upload_best", "primary")}
          </div>

          <div class="grid1" style="margin-top:10px;">
            {_btn("➡️ Continuar", "tr_to_menu", "success", disabled=(not can_continue))}
          </div>

          {log_html}
        """
        return _wrap(body)

    if scr == "training_menu":
        body = f"""
          {_topbar("Modo entrenamiento")}

          <div class="card">
            <div class="cardtitle">📌 Acciones</div>
            <div class="p">Elige una acción y se abrirá un entorno dedicado con checklist.</div>
          </div>

          <div class="grid2">
            {_btn("🎞️ Video → Frames", "tr_go_frames", "info")}
            {_btn("🧠 Entrenar / Validar", "tr_go_trainval", "warning")}
          </div>

          {log_html}
        """
        return _wrap(body)

    if scr == "training_frames":
        required = ["video"]
        can_run = (len(_missing(required)) == 0)

        body = f"""
          {_topbar("Entrenamiento — Video → Frames")}

          <div class="card">
            <div class="cardtitle">📌 Faltantes</div>
            <div class="p"><b>{_missing_text(required)}</b></div>
          </div>

          {_requirements_html(required, "📥 Checklist video")}

          <div class="grid1">
            {_btn("📤 Subir video", "tr_frames_upload", "primary")}
          </div>

          <div class="grid1" style="margin-top:10px;">
            {_btn("▶️ Extraer frames", "tr_frames_run", "success", disabled=(not can_run))}
          </div>

          {log_html}
        """
        return _wrap(body)

    if scr == "training_trainval":
        required = ["cvat"]
        can_prepare = (len(_missing(required)) == 0)

        body = f"""
          {_topbar("Entrenamiento — Preparar dataset")}

          <div class="card">
            <div class="cardtitle">📌 Faltantes</div>
            <div class="p"><b>{_missing_text(required)}</b></div>
          </div>

          {_requirements_html(required, "📥 Checklist CVAT ZIP")}

          <div class="grid1">
            {_btn("📤 Subir CVAT ZIP", "tr_tv_upload", "primary")}
          </div>

          <div class="grid1" style="margin-top:10px;">
            {_btn("📦 Preparar dataset", "tr_tv_prepare", "warning", disabled=(not can_prepare))}
          </div>

          <div class="grid1" style="margin-top:10px;">
            {_btn("➡️ Continuar", "tr_tv_continue", "success", disabled=(not (_dataset_ready() and _UI['training_ready'])))}
          </div>

          {log_html}
        """
        return _wrap(body)

    if scr == "training_runmenu":
        ready = _dataset_ready() and _UI["training_ready"]
        body = f"""
          {_topbar("Entrenamiento — Entrenar / Validar")}

          <div class="card">
            <div class="cardtitle">📌 Estado</div>
            <div class="p">Dataset: <b>{"listo" if ready else "no disponible"}</b></div>
          </div>

          <div class="grid2">
            {_btn("🏋️ Entrenar", "tr_train", "danger", disabled=(not ready))}
            {_btn("🧪 Validar", "tr_val", "success", disabled=(not ready))}
          </div>

          {log_html}
        """
        return _wrap(body)

    return _wrap(f"<div class='card'><div class='cardtitle'>Pantalla no disponible</div></div>{log_html}")

def _render():
    clear_output(wait=True)
    display(HTML(_screen_html()))

# -----------------------------
# Ejecutar motor y capturar logs (silenciando spam)
# -----------------------------
def _run_motor(step_title, fn, show_timer=False):
    buf = io.StringIO()
    start = time.time()

    try:
        with _SilenceUltralytics(), contextlib.redirect_stdout(buf), contextlib.redirect_stderr(buf):
            print("⏳", step_title)
            print("-"*80)
            fn()
            print("-"*80)
            print("✅ Finalizado:", step_title)
    except SystemExit as e:
        print("-"*80, file=buf)
        print("🟠 Proceso detenido:", str(e), file=buf)
    except Exception as e:
        print("-"*80, file=buf)
        print("❌ Error:", str(e), file=buf)
        print("\n🧾 Detalle técnico:", file=buf)
        traceback.print_exc(file=buf)

    elapsed = int(time.time() - start)
    txt = _pretty_log(buf.getvalue())
    if show_timer:
        txt = f"{txt}\n\n⏱️ Tiempo: {elapsed}s"
    _UI["log"] = txt

# -----------------------------
# Descargas: más robustas (NO re-render después)
# -----------------------------
def _download(path_str, label=None):
    try:
        from google.colab import files as colab_files
        p = Path(path_str)
        if p.exists() and p.stat().st_size > 0:
            _UI["last_download"] = str(p)
            _UI["last_download_label"] = label or p.name
            colab_files.download(str(p))
            return True
    except Exception:
        pass
    return False

# -----------------------------
# Navegación
# -----------------------------
def _go(screen_name):
    _UI["screen"] = screen_name
    _UI["nav"].append(screen_name)
    _render()

def _back():
    if len(_UI["nav"]) <= 1:
        return
    _UI["nav"].pop()
    _UI["screen"] = _UI["nav"][-1]
    _render()

# -----------------------------
# Callback principal
# -----------------------------
def daich_ui_action(action):
    # Navegación base
    if action == "go_home":
        _UI["log"] = ""
        _UI["training_ready"] = False
        _UI["nav"] = ["welcome", "home"]
        _UI["screen"] = "home"
        _render()
        return

    if action == "go_help":
        _UI["log"] = ""
        _go("help")
        return

    if action == "go_status":
        _go("status")
        return

    if action == "back":
        _back()
        return

    if action == "dl_last":
        if _UI["last_download"]:
            _download(_UI["last_download"], _UI["last_download_label"])
        return

    # -----------------------------
    # DETECCIÓN
    # -----------------------------
    if action == "go_detection":
        _UI["log"] = "📌 Modo detección listo."
        _go("detection")
        return

    if action == "det_upload":
        # Una sola subida por click (no loops)
        _run_motor("Carga de archivos (detección)", subida_archivos)
        _render()
        return

    if action == "det_run":
        required = ["best", "video", "srt", "losa", "paramento"]
        if len(_missing(required)) > 0:
            _UI["log"] = "❌ Faltan archivos. Sube lo pendiente antes de ejecutar."
            _render()
            return

        _UI["log"] = "⏳ Ejecutando detección…"
        _render()

        _run_motor("Detección + reporte (PDF)", detectar, show_timer=True)
        _render()

        _download("/content/Reporte_Inspeccion_Final.pdf", "Reporte PDF (detección)")
        return

    # -----------------------------
    # ENTRENAMIENTO
    # -----------------------------
    if action == "go_training":
        _UI["training_ready"] = False
        _UI["log"] = "📌 Sube best.pt para habilitar entrenamiento."
        _go("training_model")
        return

    if action == "tr_upload_best":
        _run_motor("Carga de best.pt", subida_archivos)
        if _presence()["best"] is not None:
            _UI["log"] = (_UI["log"] + "\n\n✅ best.pt detectado. Puedes continuar.")
        _render()
        return

    if action == "tr_to_menu":
        if _presence()["best"] is None:
            _UI["log"] = "❌ Falta best.pt."
            _render()
            return
        _UI["log"] = "✅ Modelo listo. Elige una acción."
        _go("training_menu")
        return

    if action == "tr_go_frames":
        _UI["log"] = "📌 Video → Frames."
        _go("training_frames")
        return

    if action == "tr_go_trainval":
        _UI["training_ready"] = False
        _UI["log"] = "📌 Preparar dataset desde CVAT."
        _go("training_trainval")
        return

    # VIDEO -> FRAMES
    if action == "tr_frames_upload":
        _run_motor("Carga de video", subida_archivos)
        _render()
        return

    if action == "tr_frames_run":
        if _presence()["video"] is None:
            _UI["log"] = "❌ Falta video."
            _render()
            return

        _UI["log"] = "⏳ Extrayendo frames…"
        _render()

        _run_motor("Extracción de frames", frames, show_timer=True)
        _render()

        _download("/content/out/frames.zip", "frames.zip (CVAT)")
        return

    # TRAIN/VAL: preparar dataset
    if action == "tr_tv_upload":
        _run_motor("Carga de CVAT ZIP", subida_archivos)
        _render()
        return

    if action == "tr_tv_prepare":
        if _presence()["cvat"] is None:
            _UI["log"] = "❌ Falta ZIP de CVAT (con 'cvat' en el nombre)."
            _render()
            return

        _UI["log"] = "⏳ Preparando dataset…"
        _render()

        _run_motor("Preparación dataset (CVAT → YOLO)", normalizarzip, show_timer=True)
        _UI["training_ready"] = True
        _render()
        return

    if action == "tr_tv_continue":
        if not (_dataset_ready() and _UI["training_ready"]):
            _UI["log"] = "❌ Aún no está listo el dataset."
            _render()
            return
        _UI["log"] = "✅ Dataset listo. Elige entrenar o validar."
        _go("training_runmenu")
        return

    # ENTRENAR / VALIDAR
    if action == "tr_train":
        if not (_dataset_ready() and _UI["training_ready"]):
            _UI["log"] = "❌ Dataset no listo."
            _render()
            return

        _UI["log"] = "⏳ Entrenando…"
        _render()

        _run_motor("Entrenamiento", entrenar, show_timer=True)
        _render()

        _download("/content/in/best.pt", "best.pt (actualizado)")
        return

    if action == "tr_val":
        if not (_dataset_ready() and _UI["training_ready"]):
            _UI["log"] = "❌ Dataset no listo."
            _render()
            return

        n_val = len(list((PATHS["dataset"]/ "images/val").glob("*")))
        _UI["log"] = f"⏳ Validando… (val imgs: {n_val})"
        _render()

        _run_motor("Validación visual", validar, show_timer=True)
        _render()

        _download("/content/out/val_predictions_clean.zip", "val_predictions_clean.zip")
        return

    _UI["log"] = f"Acción no reconocida: {action}"
    _render()

output.register_callback("daich_ui_action", daich_ui_action)

def iniciar_panel_daich_html():
    _UI["screen"] = "welcome"
    _UI["nav"] = ["welcome"]
    _UI["log"] = ""
    _UI["training_ready"] = False
    _render()

iniciar_panel_daich_html()
